In [45]:
import torch
import torch.nn as nn
from torch.nn import functional as F
device = 'cuda' if torch.cuda.is_available() else 'cpu'
block_size = 2**3
batch_size = 2**2
learning_rate = 3e-4
max_epoch = int(1e4)
eval_interval = 250
dropout = 0.2

In [46]:
with open("wizard_of_oz.txt", 'r', encoding="utf-8") as f:
    text = f.read()
chars = sorted(set(text))
print(chars)
vocab_size = len(chars)

['\n', ' ', '!', '"', '&', "'", '(', ')', '*', ',', '-', '.', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '?', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '\ufeff']


In [47]:
string_to_int = { ch:i for i,ch in enumerate(chars) }
int_to_string = { i:ch for i,ch in enumerate(chars) }
encode = lambda s: [string_to_int[c] for c in s]
decode = lambda l: ''.join([int_to_string[i] for i in l])

data = torch.tensor(encode(text), dtype=torch.long)
print(data[:100])

tensor([80,  1,  1, 28, 39, 42, 39, 44, 32, 49,  1, 25, 38, 28,  1, 44, 32, 29,
         1, 47, 33, 50, 25, 42, 28,  1, 33, 38,  1, 39, 50,  0,  0,  1,  1, 26,
        49,  0,  0,  1,  1, 36, 11,  1, 30, 42, 25, 38, 35,  1, 26, 25, 45, 37,
         0,  0,  1,  1, 25, 45, 44, 32, 39, 42,  1, 39, 30,  1, 44, 32, 29,  1,
        47, 33, 50, 25, 42, 28,  1, 39, 30,  1, 39, 50,  9,  1, 44, 32, 29,  1,
        36, 25, 38, 28,  1, 39, 30,  1, 39, 50])


In [48]:
n = int(0.8*len(data))
train_data = data[:n]
val_data = data[n:]

def get_batch(split):
    data = train_data if split =="train" else val_data
    ix = torch.randint(len(data) - block_size, (batch_size,))
    
    x = torch.stack([data[i:i+block_size] for i in ix])
    y = torch.stack([data[i+1:i+block_size+1] for i in ix])
    x, y = x.to(device), y.to(device)
    return x,y

x, y = get_batch('train')
print('input:')
print(x)
print('target:')
print(y)

input:
tensor([[ 1, 68, 67, 58,  0, 68, 59,  1],
        [ 1, 33,  1, 76, 54, 67, 73, 58],
        [71, 62, 58, 67, 57, 65, 78,  1],
        [ 1, 73, 61, 58,  1, 65, 62, 73]], device='cuda:0')
target:
tensor([[68, 67, 58,  0, 68, 59,  1, 73],
        [33,  1, 76, 54, 67, 73, 58, 57],
        [62, 58, 67, 57, 65, 78,  1, 54],
        [73, 61, 58,  1, 65, 62, 73, 73]], device='cuda:0')


In [49]:
@torch.no_grad()
def estimate_loss():
    out = {}
    m.eval()
    for split in ['train', 'val']:
        losses = torch.zeros(eval_interval)
        for k in range(eval_interval):
            X, Y = get_batch(split)
            logits, loss = m.forward(X,Y)
            losses[k] = loss.item()
        out[split] = losses.mean()
    m.train()
    return out

In [50]:
class BigramLanguageModel(nn.Module):
    def __init__(self, vocab_size):
        super().__init__()

        self.token_embedding_table = nn.Embedding(vocab_size, vocab_size)

    def forward(self, idx, targets=None):
        logits = self.token_embedding_table(idx)

        if targets is None:
            loss = None
        else:
            B, T, C = logits.shape
            logits = logits.view(B*T, C)
            targets = targets.view(B*T)
            loss = F.cross_entropy(logits, targets)

        return logits, loss

    def generate(self, idx, max_new_tokens):
        # index is (B, T) array of indices in the current context
        for _ in range(max_new_tokens):
            # get the predictions
            logits, loss = self.forward(idx)
            # focus only on the last time step
            logits = logits[:, -1, :] # becomes (B, C)
            # apply softmax to get probabilities
            probs = F.softmax(logits, dim=-1) # (B,C)
            # sample from the distribution
            idx_next = torch.multinomial(probs, num_samples=1) # (B, 1)
            # append sampled index to the running sequence
            idx = torch.cat((idx, idx_next), dim=1) # (B, T+1)
        return idx

model = BigramLanguageModel(vocab_size)
m = model.to(device)

context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


?7[0[fso1'
;WeW?pH!lyWaRI0Xaxl2eMX3Btr1X0NJDI5e-OpGqW*uj75t5.!DGr﻿ryiuy]DdRq3&5cn)y?7P3﻿0CYg'9c
[KEz4i
!(kjEqo4Qv'!oAnKc
q75pN!0NJJRS]URFVbU8YUx;tTx;W3'8!D4K9s7M 2n)yqEdUkG﻿cPFY*qWre4zL2(A!aWN3oNJ15c;:gsIN!PL&,Or:a5.exZI?pzOBla
7
Te[b)N!OKd*ZVDG_Q'p!)C3'EZ,Uk N*5ohn)8j8Je[tbrqo8cttF]l[C6_0;,
7DKhXtrN**:iu,?lp]u-.TS*&n"H[wQDGrE(F;WHA!IYmecle[,h,6P(t0RKV[MX24g;Q,'E"OYWo8YXM:]w:CVzf]f SCG;uw;3'7P]at-ZVEHtrCmarXy[76Z99qYmCoF]DGbAAC.T
S9(yi*9eW4wkAHK﻿rY(wzO37s1P;i!gDGJczO].;7nS[b:qRdYjqnDG.3hsh[b81H



In [56]:
optimizer = torch.optim.AdamW(model.parameters(), lr=learning_rate)

for epoch in range(max_epoch):

    if epoch % eval_interval == 0:
        losses = estimate_loss()
        print(f"step: {epoch}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

    # sample avbatch of data
    xb, yb = get_batch('train')

    # evaluate the loss
    logits, loss = m.forward(xb, yb)
    optimizer.zero_grad(set_to_none=True)
    loss.backward()
    optimizer.step()
print(f"step: {epoch}, train loss: {losses['train']:.4f}, val loss: {losses['val']:.4f}")

step: 0, train loss: 2.6826, val loss: 2.7054
step: 250, train loss: 2.6744, val loss: 2.7136
step: 500, train loss: 2.6777, val loss: 2.6941
step: 750, train loss: 2.6705, val loss: 2.7089
step: 1000, train loss: 2.6419, val loss: 2.7025
step: 1250, train loss: 2.6330, val loss: 2.6917
step: 1500, train loss: 2.6474, val loss: 2.6843
step: 1750, train loss: 2.6078, val loss: 2.6705
step: 2000, train loss: 2.6276, val loss: 2.6571
step: 2250, train loss: 2.6121, val loss: 2.6804
step: 2500, train loss: 2.6218, val loss: 2.6703
step: 2750, train loss: 2.5997, val loss: 2.6552
step: 3000, train loss: 2.5995, val loss: 2.6293
step: 3250, train loss: 2.6066, val loss: 2.6588
step: 3500, train loss: 2.6133, val loss: 2.6281
step: 3750, train loss: 2.6096, val loss: 2.6251
step: 4000, train loss: 2.6101, val loss: 2.6330
step: 4250, train loss: 2.5970, val loss: 2.6290
step: 4500, train loss: 2.5832, val loss: 2.6316
step: 4750, train loss: 2.5687, val loss: 2.6320
step: 5000, train loss: 2.

In [57]:
context = torch.zeros((1,1), dtype=torch.long, device=device)
generated_chars = decode(m.generate(context, max_new_tokens=500)[0].tolist())
print(generated_chars)


begs
NFWis, ub hererdlculsFpsk resanq92k?j-Hace sore
I t be bo. suthya caned mer eag thedendound he GXQqWerdepots JzWinde in
"
vedy tethen ofrd theagik
"m. mo  himen lizqn t bormus hinuly, waswalirs.
acilane d od wond ors 8dd uremm,  car, 'rn.rney.  pad foris.
"AQDised ts b,Bgrer h sir fu'ple, ofllen thed creroelo co ad:ared edod



"oofofun iere owls inisthepaserWh
Tbe ssmad:veanvy67ccoimbyo opl byot theyolsth n  te in bblesit aitun, inogebry, ET43I
he r, tt." ch.

HLan
or withigevqIC&chthese M
